In [20]:
import os
import subprocess
from bs4 import BeautifulSoup

In [21]:
path_default = "data/"
path_target = "articles.A-B.xml"
path_opennlp = "apache-opennlp-1.8.1/"

In [22]:
labels = {}
file_label = open(path_default + 'Structured-Abstracts-Labels-102615.txt', 'r')
for line in file_label.readlines() :
    label = line.split('|')
    labels[label[0]] = label[1]
    
#labels

In [23]:
comment = 'Two symptom-based screening questionnaires, the Berlin Questionnaire (BQ) and the Epworth Sleepiness Scale (ESS), were administered to enroll 1509 gravidae. Screen positive subjects were referred for polysomnography (PSG). The primary outcome was the occurrence of either gestational hypertension or preeclampsia. [a1]Generalized linear models (GLM) were used to estimate the relative risks of associations.'
process_nlp = subprocess.Popen([path_opennlp + 'bin/opennlp',  'SentenceDetector', path_opennlp + 'en-sent.bin'], stdin = subprocess.PIPE, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
result, stderr = process_nlp.communicate(comment.encode())
for line in result.decode().split('\n') :
    print(line)

Two symptom-based screening questionnaires, the Berlin Questionnaire (BQ) and the Epworth Sleepiness Scale (ESS), were administered to enroll 1509 gravidae.
Screen positive subjects were referred for polysomnography (PSG).
The primary outcome was the occurrence of either gestational hypertension or preeclampsia.
[a1]Generalized linear models (GLM) were used to estimate the relative risks of associations.

Execution time: 0.104 seconds

